<a href="https://colab.research.google.com/github/inverter404/bosch-interiit/blob/main/test_greybox_k400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test folder is strutured as follows:

    k400
        -annotations
            -testwithlabels.csv
        -test
            -vid1
            -vid2
            -vid3

**IMPORTING LIBRARIES**

In [ ]:
import os
import sys

import cv2

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

from tqdm import tqdm

import torch
import torch.nn.parallel
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.optim
from torch.nn.utils import clip_grad_norm_
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader


**DEFINING TDN MODEL**

In [ ]:
!git clone https://github.com/MCG-NJU/TDN 

In [ ]:
sys.path.append('/content/TDN')

In [ ]:
from ops.models import TSN

In [ ]:
thief_model = TSN(400,3,'RGB',                                  
                base_model='resnet101',
                consensus_type='avg',
                dropout=0.5,
                img_feature_dim=256,
                partial_bn=False,
                pretrain=False,
                fc_lr5=False)

**LOADING THE LEARNED WEIGHTS**

In [ ]:
device=torch.device('cuda:0')
checkpoint=torch.load('/content/drive/MyDrive/Action Recognition/weights_400',map_location='cuda:0')
thief_model.load_state_dict(checkpoint['state_dict'])
thief_model=thief_model.to(device)

**LOADING THE CSV AND ENCODING THE LABELS**

In [ ]:
meta=pd.read_csv('/content/drive/MyDrive/k400/annotations/testwithlabels.csv')

In [ ]:
meta

,label,youtube_id,time_start,time_end,split,id
0,sipping cup,--0l35AkU34,62,72,test,461
1,using inhaler,--71SekUwOA,5,15,test,561
2,climbing tree,--8YXc8iCt8,2,12,test,102
3,washing feet,--GkrdYZ9Tc,0,10,test,574
4,playing kickball,--NFqQGeShc,33,43,test,351
...,...,...,...,...,...,...
56613,wading through mud,zzfd7GRadHc,0,10,test,567
56614,popping balloons,zzkCQkQvy04,18,28,test,383
56615,jumping into pool,zzpqbqLllzA,1,11,test,241
56616,using bagging machine,zzz0X4NoFyA,60,70,test,559


In [ ]:
oe=OrdinalEncoder()
oe.fit(meta[['label']])
meta[['label']]=oe.transform(meta[['label']])
meta['label']=meta['label'].astype('int')

In [ ]:
meta

,label,youtube_id,time_start,time_end,split,id
0,461,--0l35AkU34,62,72,test,461
1,561,--71SekUwOA,5,15,test,561
2,102,--8YXc8iCt8,2,12,test,102
3,574,--GkrdYZ9Tc,0,10,test,574
4,351,--NFqQGeShc,33,43,test,351
...,...,...,...,...,...,...
56613,567,zzfd7GRadHc,0,10,test,567
56614,383,zzkCQkQvy04,18,28,test,383
56615,241,zzpqbqLllzA,1,11,test,241
56616,559,zzz0X4NoFyA,60,70,test,559


**STORING VIDEO PATHS AND CORRESPONDING LABELS**

In [ ]:
video_path=[]
labels=[]
'''
labels of videos are found and stored in the labels list
'''
for name in tqdm(os.listdir('/content/drive/MyDrive/k400/test/')):
  nam=name[0:11]
  dir='/content/drive/MyDrive/k400/test/'+name
  lab=meta.loc[(meta['youtube_id']==nam)]      # searching for label of video in meta using the youtube_id
  vid=cv2.VideoCapture(dir)
  length=(int(vid.get(cv2.CAP_PROP_FRAME_COUNT)))
  if(length<17):
    continue
  elif(lab.empty):
    continue
  lab=lab.head(1)
  classe=lab['label'].item()
  labels.append(classe)
  video_path.append(dir)


**DATASET (TAKES 16 EQUALLY SPACED FRAMES FROM VIDEOS)**

In [ ]:
class Vid_Dataset(Dataset):
    '''
    creates dataset of videos from k400 folder
    16 frames are extracted from each video
    labels are the ground truth
    '''
    def __init__(self,video_path,labels):
        self.n_samples=labels.shape[0]
        self.video_path=video_path
        self.labels=labels
    def __getitem__(self, index):
        vid=cv2.VideoCapture(self.video_path[index])
        array=[]
        success = 1
        count=0
        cnt=0
        length=(int(vid.get(cv2.CAP_PROP_FRAME_COUNT)))//16
        while success:
            success, image = vid.read()
            count += 1
            if(count%length==0 and cnt<16):
                cnt+=1
                array.append(np.array(image))
        array=np.array(array)
        array=array.astype('float32')
        sample=(array,self.labels[index])
        return sample
    def __len__(self):
        return self.n_samples

**MAKING DATALOADER**

In [ ]:
dataset2=Vid_Dataset(video_path,labels)
val_loader=DataLoader(dataset=dataset2,batch_size=1,shuffle=False)

**TEST FUNCTION**

In [ ]:
def test(model):
    '''
    prints top-5 accuracy after every 100 videos 
    and finally prints the overall top-5 accuracy
    '''
    thief_model.eval()
    with torch.no_grad():
        cnt=0
        num=0
        for inputs,labels in tqdm((val_loader)):
            inputs=np.squeeze(inputs)
            inputs=inputs.permute(3,0,1,2)
            inputs=inputs.to(device)

            labels=labels.to(device)
            out=model(inputs,no_reshape=True)
            pred=torch.argsort(out[0],descending=True)
            for j in range(5):
                if(pred[j].item()==labels[0].item()):
                    cnt+=1
                    break
            num+=1
            if(num%100==0):
                print(f'Validation Accuracy: {(cnt/num)*100}')
    print(f'Final Validation Accuracy: {(cnt/num)*100}')

**P1 GREYBOX ACCURACY**

In [ ]:
test(thief_model)